#### Contains a few useful functions.

find_games_played - takes a csv of matches, and for each god, creates a list of the matches (as int) that that god won or lost.
                    Then saves all lists in 1 dictionary, keyed:  god_id + "_win"/"_lose" : [], output as a json.
                    Creates file in in SMITE_data, which is necessary for SmiteRecommender to initialize

stitch          - stitches together all the parsed matches from a folder and creates one csv file to be used by SmiteRecommender.
                    (find_games_played also takes in the output of stitcher)


In [2]:
import os, json
import pandas as pd

In [3]:
def find_games_played(gamemode):
    """ writes a dictionary (as a json file) with a list of played matches keyed to each god
    each god has two keys: god_id + '_win', and god_id + '_lose'
    Uses SMITE_Gods_All_Lookup.csv, assuming that it is the SMITE_data folder.
    standard output name: 'matches_played_by_each_god_ranked.json'
    gamemodes:
    "ranked" (ranked conquest)
    "casual" (casual conquest)
    "joust"  (ranked joust)
    "duel"   (ranked duel)
    """
    if gamemode == "ranked":
        datafile   = 'Ranked_Conquest_SMITEdata.csv'
        outputfile = "matches_played_by_each_god_ranked.json"
    elif gamemode == "casual":
        datafile   = 'Casual_Conquest_SMITEdata.csv'
        outputfile = "matches_played_by_each_god_casual.json"
    elif gamemode == "joust":
        datafile   = 'Ranked_Joust_SMITEdata.csv'
        outputfile = "matches_played_by_each_god_joust.json"
    elif gamemode == "duel":
        datafile   = 'Ranked_Duel_SMITEdata.csv'
        outputfile = "matches_played_by_each_god_duel.json"
    else:
        print("ERROR: gamemode not valid. Choose ranked, casual, joust, or duel.")
        return
                        
    df = pd.read_csv(datafile).set_index('match_id')
    # make a list of all god_ids
    god_ids = pd.read_csv("SMITE_Gods_All_Lookup.csv").id.tolist()
    each_god_matches = {}
    for god_id in god_ids:
        matches = df[df['god_id']==god_id]
        # go to great lengths to make sure the dict values are int.             
        each_god_matches[str(god_id) + '_win'] = matches.index[matches['win'] == True].get_values().tolist()
        each_god_matches[str(god_id) + '_lose'] = matches.index[matches['win'] == False].get_values().tolist()
    with open(outputfile, 'w') as fout:
        json.dump(each_god_matches, fout)
    print('Game matching complete: "{}" created.'.format(outputfile))

In [4]:
def stitch(gamemode):
    """ stiches together all match files for the given gamemode and creates new file to be used by SmiteRecommender
    gamemodes:
    "ranked" (ranked conquest)
    "casual" (casual conquest)
    "joust"  (ranked joust)
    "duel"   (ranked duel)
    """
    if gamemode == "ranked":
        outputfile = 'Ranked_Conquest_SMITEdata.csv'
        path       = '../SMITE_scraped_parsed_ranked/'
    elif gamemode == "casual":
        outputfile = 'Casual_Conquest_SMITEdata.csv'
        path       = '../SMITE_scraped_parsed_casual/'
    elif gamemode == "joust":
        outputfile = 'Ranked_Joust_SMITEdata.csv'
        path       = '../SMITE_scraped_parsed_joust/'
    elif gamemode == "duel":
        outputfile = 'Ranked_Duel_SMITEdata.csv'
        path       = '../SMITE_scraped_parsed_duel/'
    else:
        print("ERROR: gamemode not valid. Choose ranked, casual, joust, or duel.")
        return
    all_files = os.listdir(path) 
    df_from_each_file = (pd.read_csv(path + f) for f in all_files) # generator (could use list, this is just a bit faster)
    concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
    concatenated_df.to_csv(outputfile, index=None)
    print('Stitch complete:"{}" created.'.format(outputfile))

In [5]:
#stitch(gamemode = "ranked")
#find_games_played(gamemode = 'ranked')

stitch(gamemode = "duel")
find_games_played(gamemode = 'duel')

#stitch(gamemode = "joust")
#find_games_played(gamemode = 'joust')

Stitch complete:"Ranked_Duel_SMITEdata.csv" created.
Game matching complete: "matches_played_by_each_god_duel.json" created.
